In [ ]:
import numpy as np
import random

def rastrigin_function(x):
    """
    Rastrigin function for optimization.
    x: List or array of genes (real numbers)
    """
    A = 10
    return A * len(x) + sum([(xi ** 2) - A * np.cos(2 * np.pi * xi) for xi in x])

def levy_flight(Lambda, size):
    """
    Generate Lévy flight step sizes.
    Lambda: Parameter controlling the Lévy distribution
    size: Size of the position vector
    """
    sigma_u = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
               (np.math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.normal(0, sigma_u, size)
    v = np.random.normal(0, 1, size)
    step = u / (abs(v) ** (1 / Lambda))
    return step

def initialize_nests(num_nests, dimensions, lower_bound, upper_bound):
    """
    Initialize a population of nests with random positions.
    num_nests: Number of nests
    dimensions: Dimensionality of the problem
    lower_bound, upper_bound: Bounds for each variable
    """
    return np.random.uniform(lower_bound, upper_bound, (num_nests, dimensions))

def evaluate_fitness(positions):
    """
    Evaluate the fitness of nests.
    positions: Array of nest positions
    """
    return np.array([rastrigin_function(pos) for pos in positions])

def abandon_worst_nests(positions, pa, lower_bound, upper_bound):
    """
    Abandon a fraction of the worst nests and replace with new random nests.
    positions: Array of current nests
    pa: Probability of abandonment
    lower_bound, upper_bound: Bounds for each variable
    """
    new_positions = positions.copy()
    for i in range(len(positions)):
        if random.random() < pa:
            new_positions[i] = np.random.uniform(lower_bound, upper_bound, len(positions[i]))
    return new_positions

def cuckoo_search(num_nests, dimensions, lower_bound, upper_bound, pa, max_iterations):
    """
    Cuckoo Search Algorithm.
    num_nests: Number of nests
    dimensions: Number of variables in the optimization problem
    lower_bound, upper_bound: Variable bounds
    pa: Probability of abandoning nests
    max_iterations: Number of iterations
    """
    # Step 1: Initialize nests
    nests = initialize_nests(num_nests, dimensions, lower_bound, upper_bound)
    fitness = evaluate_fitness(nests)

    # Track the best nest
    best_nest = nests[np.argmin(fitness)]
    best_fitness = min(fitness)

    for iteration in range(max_iterations):
        new_nests = nests.copy()
        # Step 2: Generate new solutions via Lévy flights
        for i in range(num_nests):
            step = levy_flight(1.5, dimensions)
            new_nests[i] = nests[i] + step * (nests[i] - best_nest)
            new_nests[i] = np.clip(new_nests[i], lower_bound, upper_bound)

        new_fitness = evaluate_fitness(new_nests)

        # Step 3: Replace nests if new solutions are better
        for i in range(num_nests):
            if new_fitness[i] < fitness[i]:
                nests[i] = new_nests[i]
                fitness[i] = new_fitness[i]

        # Step 4: Abandon worst nests
        nests = abandon_worst_nests(nests, pa, lower_bound, upper_bound)
        fitness = evaluate_fitness(nests)

        # Update the best nest
        current_best_index = np.argmin(fitness)
        if fitness[current_best_index] < best_fitness:
            best_nest = nests[current_best_index]
            best_fitness = fitness[current_best_index]

        print(f"Iteration {iteration + 1}: Best Fitness = {best_fitness:.4f}")

    print("Optimization Complete!")
    return best_nest, best_fitness

# Parameters
NUM_NESTS = 30  # Number of nests
DIMENSIONS = 5  # Dimensionality of the problem
LOWER_BOUND = -5.12
UPPER_BOUND = 5.12
PROBABILITY_ABANDON = 0.25  # Probability of abandoning a nest
MAX_ITERATIONS = 5

# Run Cuckoo Search Algorithm
best_solution, best_fitness = cuckoo_search(NUM_NESTS, DIMENSIONS, LOWER_BOUND, UPPER_BOUND, PROBABILITY_ABANDON, MAX_ITERATIONS)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)


Iteration 1: Best Fitness = 54.0829
Iteration 2: Best Fitness = 50.2743
Iteration 3: Best Fitness = 38.1236
Iteration 4: Best Fitness = 38.1236
Iteration 5: Best Fitness = 38.1236
Optimization Complete!
Best Solution: [-0.91440462 -0.67534105 -1.82405393 -0.7255691   0.00398268]
Best Fitness: 38.123631579961156


<ipython-input-1-aa0e0b2a9aac>:18: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  sigma_u = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
<ipython-input-1-aa0e0b2a9aac>:19: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  (np.math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
